# **ImpressionGPT**

package install

In [ ]:
!pip install -q openai
!pip install -q evaluate rouge_score
!pip install -q rouge

In [ ]:
import pandas as pd
import os
import openai
import concurrent.futures
from tqdm import tqdm
import evaluate
import numpy as np
import time
from rouge import Rouge
import rouge

# This is your key
openai.api_key = "This is your key"
print(openai.api_key)

# GPT_api_dynamic_prompt_with_interactive

In [ ]:
def chestGPT_summary_near_sample_with_interactive_v6(text, near_samples=None, interactive=False, former_good_response=None, former_bad_response=None):
  # One good response and One Bad response.
  max_tries = 2

  dynamic_messages = [
      {"role": "system", "content": "You are a chest radiologist that identify the main findings and diagnosis or impression based on the given\
        FINDINGS section of the chest X-ray report, which details the radiologists' assessment of the chest X-ray image. \
        Please ensure that your response is concise and does not exceed the length of the FINDINGS."}]
  if near_samples is not None:
    dynamic_messages.append({"role": "user", "content": "Here are some examples. Please learn how to write IMPRESSION in these examples, and \
    pay particular attention to the consistent use of phrases in these below examples."})
    for near in near_samples:
      dynamic_messages.append({"role": "user", "content": "What are the main findings and diagnosis or impression based on the given Finding in chest X-ray report:\
          \nFINDINGS:\n{}".format(near["finding"])})
      dynamic_messages.append({"role": "assistant", "content": "IMPRESSION:\n{}".format(near["impression"])})

  for try_number in range(max_tries):
    try:
      if not interactive:
        dynamic_messages.append({"role": "user", "content": "What are the main findings and diagnosis or impression based on the given Finding in \
          chest X-ray report: \nFINDINGS: {}".format(text)})
        # print(dynamic_messages)
        response = openai.ChatCompletion.create(
          # model="gpt-4",
          model="gpt-3.5-turbo",
          messages=dynamic_messages )
      else:
        dynamic_messages.append({"role": "user", "content": "What are the main findings and diagnosis or impression based on the given Finding in chest \
                        X-ray report: \nFINDINGS:\n{}".format(text)})
        if former_bad_response is not None:
          for index, item in enumerate(former_bad_response):
            dynamic_messages.append({"role": "user", "content": "Below is a bad impression of the FINDINGS above:"})
            dynamic_messages.append({"role": "assistant", "content": "IMPRESSION:\n{}".format(item["summarize"])})

          if len(former_good_response) == 0:
            dynamic_messages.append({"role": "user", "content": "Please give another impression of the FINDINGS above. \
            It is important to note that your answer should avoid being consistent with the bad impression above. \
            Please pay particular attention to the consistent use of phrases in the above examples"})
          # else:
          #   dynamic_messages.append({"role": "user", "content": "Please give another impression based on the above FINDINGS.\
          #         Try to refer to the good response above and avoid the same bad response as above."})
        
        if former_good_response is not None:
          for index, item in enumerate(former_good_response):
            dynamic_messages.append({"role": "user", "content": "Below is an excellent impression of the FINDINGS above:"})
            dynamic_messages.append({"role": "assistant", "content": "IMPRESSION:\n{}".format(item["summarize"])})
            # dynamic_messages.append({"role": "user", "content": "This is a excellent impression, please give another clearer impression in this format."})
          if len(former_bad_response) == 0:
            dynamic_messages.append({"role": "user", "content": "This is an excellent impression, \
              please give another impression in this format, and do not exceed the length of this impression.\
              Please pay particular attention to the consistent use of phrases in the above examples"})
          else:
            dynamic_messages.append({"role": "user", "content": "Please give another impression of the FINDINGS above. \
              It is important to note that your answer should avoid being consistent with the bad impression above, \
              but should be consistent with the excellent impression above, and do not exceed the length of the excellent impression. \
              And please pay particular attention to the consistent use of phrases in the above examples"})


        # print(dynamic_messages)
        # time.sleep(20)
        response = openai.ChatCompletion.create(
          # model="gpt-4",
          model="gpt-3.5-turbo",
          messages=dynamic_messages
        )
        # time.sleep(20)
      return response['choices'][0]['message']['content']

    except openai.error.APIError as e:
      if try_number == max_tries - 1:
        print('APIError')
        return '\n'
      else:
        time.sleep(0.1)
    except openai.error.Timeout as e:
      if try_number == max_tries - 1:
        print('Timeout')
        return '\n'
      else:
        time.sleep(0.1)
    except openai.error.APIConnectionError as e:
      if try_number == max_tries - 1:
        print('APIConnectionError')
        return '\n'
      else:
        time.sleep(0.1)
    except openai.error.RateLimitError as e:
      if try_number == max_tries - 1:
        print('RateLimitError')
        return '\n'
      else:
        time.sleep(0.1)
    except openai.error.InvalidRequestError as e:
      if try_number == max_tries - 1:
        print('InvalidRequestError')
        return -2
      else:
        time.sleep(0.1)

# random_sample_selection_v2

similar search

In [ ]:
def random_sample_selection_v2(test_sample_label, label_space, m=2000, k=20):
  # MIMIC-CXR
  dist = []
  # study_ids = []
  randon_choice_corpus = label_space
  # randon_choice_corpus = label_space.sample(n=m)
  for index, row in randon_choice_corpus.iterrows():
    # print(row[2:])
    # if (np.array(row[2:]) == example_label).all() and row["study_id"] != test_study_id:
    label = np.array(row[2:])
    # label[np.isnan(label)] = 2
    eucliDist = np.sqrt(sum(np.power((label - test_sample_label), 2)))
    dist.append(eucliDist)
    # study_ids.append(row["study_id"])
    # if np.allclose(np.array(row[2:]), example_label) and row["study_id"] != test_study_id:
    #   same_samples.append(row["study_id"])
  
  np_dist_scores = np.array(dist)
  sort_index = np.argsort(np_dist_scores)

  bottom_index = sort_index[:k]
  best_examples, best_dist, best_label = [], [], []
  for index in bottom_index:
    # if randon_choice_corpus.iloc[index]["study_id"] != study_ids[index]:
    #   print(randon_choice_corpus.iloc[index]["study_id"], study_ids[index], index)
      
    best_examples.append(randon_choice_corpus.iloc[index]["study_id"])
    best_dist.append(dist[index])
    # best_label.append(np.array(randon_choice_corpus.iloc[index][2:]))

  # print(best_dist)
  return best_examples
 

In [ ]:
def random_sample_selection_v2_for_openI(test_sample_label, label_space, m=2000, k=20):
  # OpenI
  dist = []
  randon_choice_corpus = label_space

  for index, row in randon_choice_corpus.iterrows():
    # print(row[2:])
    label = np.array(row[1:])
    eucliDist = np.sqrt(sum(np.power((label - test_sample_label), 2)))
    dist.append(eucliDist)

  np_dist_scores = np.array(dist)
  sort_index = np.argsort(np_dist_scores)

  bottom_index = sort_index[:k]
  best_examples, best_dist, best_label = [], [], []
  for index in bottom_index:

    best_examples.append(randon_choice_corpus.iloc[index]["ids"])
    best_dist.append(dist[index])

  return best_examples
 

# Test One Csv

only compute local csv

In [ ]:
import os.path as osp
import math as Math
import pandas as pd

df_test_csv = pd.read_csv("Local CSV Path")
rouge = Rouge()

rouge_e = evaluate.load('rouge')

rouge_scores = []
for index, row in df_test_csv.iterrows():
    label = row['impression\n']
    prediction = row['summary']
    prediction = prediction.replace("IMPRESSION:", "")
    score = rouge_e.compute(predictions=[prediction], references=[label], rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores.append(score)


# mean
mean_rouge1 = sum([score['rouge1'] for score in rouge_scores]) / len(rouge_scores)
mean_rouge2 = sum([score['rouge2'] for score in rouge_scores]) / len(rouge_scores)
mean_rougeL = sum([score['rougeL'] for score in rouge_scores]) / len(rouge_scores)

print(f'R-1: {mean_rouge1:.4f}', f'R-2: {mean_rouge2:.4f}', f'R-L: {mean_rougeL:.4f}')




# one good and one bad
test MIMIC

In [ ]:
from rouge import Rouge
import os.path as osp
import os
rouge = Rouge()


root = "Root path of saving CSV"

# MIMIC-CXR offical training split.
train_data = pd.read_csv("Path of mimic_report_sections_train_v4.csv") 
# CheXpert is used to obtain the train data labels.
train_label_space = pd.read_csv('Path of train_v4_labels.csv')

# MIMIC-CXR offical testing split. CheXpert is used to obtain the label.
test_label_space = pd.read_csv('Path of test_v6_labels.csv')



test_label_study_ids = []
for index, row in test_label_space.iterrows():
  test_label_study_ids.append(row["study_id"])


# Pre-defined para
interactive = True
near_k_samples = 15
interactive_times = 17
rouge_thre = 0.7
rouge_type = "rouge-1"

"""
Below is the best paras for now:
interactive = True
near_k_samples = 15
interactive_times = 17
rouge_thre = 0.7
rouge_type = "rouge-1"

"""

def process_row_with_random_choice_samples(args):
    row_index, row = args

    test_subject_id, test_study_id = row['subject_id'], row['study_id']
    test_findings, test_impression = row["finding"], row["impression"]

    # Test label vector
    test_sample_label = np.array(test_label_space.iloc[test_label_study_ids.index(test_study_id)][2:])

    # Similar Search. Return Top-near_k_samples samples' id.  m is the size of our corpus. randomly sample from all training data.
    near_study_ids = random_sample_selection_v2(test_sample_label, train_label_space, m=10000, k=near_k_samples)

    # Get the similar report Findings and Impression based on 'near_study_ids'
    near_samples = []
    for index, train_row in train_data.iterrows():
      if len(near_study_ids) == 0:
        break
      if int(train_row["study_id"]) in near_study_ids:
        near_samples.append({"finding": train_data.iloc[index]["finding"], "impression": train_data.iloc[index]["impression"]})
        near_study_ids.remove(train_row["study_id"])

    good_reponse = []
    bad_reponse = []
    former_socre = 0
    former_bad_socre = 0
    all_response_score, all_response = [], []
    try_count = 0
    while True:
      if len(good_reponse) == 0 and len(bad_reponse) == 0:
        reponse = chestGPT_summary_near_sample_with_interactive_v4(test_findings, near_samples=near_samples)
      else:
        reponse = chestGPT_summary_near_sample_with_interactive_v4(test_findings, near_samples=near_samples, interactive=interactive, former_good_response=good_reponse, former_bad_response=bad_reponse)
     
      fotmatted_response = reponse.replace("IMPRESSION:", "")

      # evaluate response
      compare_scores = []
      for near_sa in near_samples:
        scores = rouge.get_scores([fotmatted_response], [near_sa["impression"]])
        single_score = scores[0][rouge_type]["f"]
        compare_scores.append(single_score)
      # print(compare_scores)
      score = np.mean(np.array(compare_scores))

      all_response_score.append(score)
      all_response.append(fotmatted_response)
      try_count += 1

      # if it is good than former
      if score >= rouge_thre and score > former_socre:
        former_socre = score
        good_reponse.clear() # clear the list after use evaluted response
        good_reponse.append({"summarize": fotmatted_response, "score": score})

      # if it is bad than former
      if score < rouge_thre - 0.1 and score < former_socre:
        former_bad_socre = score
        bad_reponse.clear()
        bad_reponse.append({"summarize": fotmatted_response, "score": score})
      if try_count > interactive_times:
        break

    # Finally choose a best score response
    max_score_index = all_response_score.index(max(all_response_score))
    max_score = all_response_score[max_score_index]
    final_best_response = all_response[max_score_index]
    # print("max score=", max_score, "\n", final_best_response)
    # print("max score=", max_score, "\n")
    return row_index, final_best_response

# mid_path is the experiment name
mid_path = "testV6_IntactV4_near{}_Ttrain_i{}r{}_{}".format(near_k_samples, interactive_times, rouge_thre, rouge_type)
save_root = osp.join(root, mid_path)
if not osp.exists(save_root):
  os.makedirs(save_root) 
print(save_root)
for loop_index in range(7):
  print(time.strftime("%Y-%m-%d %H:%M:%S"))
  Test_start = loop_index * 200
  Test_end = (loop_index + 1) * 200
  print(Test_start, Test_end)
  df_test_csv = pd.read_csv('Path of mimic_report_sections_test_v6.csv')
  df_test_csv = df_test_csv[Test_start:Test_end]

  # parallel
  with concurrent.futures.ThreadPoolExecutor() as executor:
    # time.sleep(20)
    results = dict(tqdm(executor.map(process_row_with_random_choice_samples, df_test_csv.iterrows()), total=len(df_test_csv)))

  # same order as the input
  df_test_csv['summary'] = df_test_csv.index.map(results.get)

  # out_file_name is name of split csv
  out_file_name = "test{}_{}V6_IntactV4_near{}_Ttrain_i{}r{}_{}.csv".format(Test_start, Test_end, near_k_samples, interactive_times, rouge_thre, rouge_type)
  output_file = osp.join(save_root, "{}".format(out_file_name))
  df_test_csv.to_csv(output_file, index=False)


  rouge_e = evaluate.load('rouge')

  rouge_scores = []
  for index, row in df_test_csv.iterrows():
      label = row['impression']
      prediction = row['summary']
      prediction = prediction.replace("IMPRESSION:", "")
      score = rouge_e.compute(predictions=[prediction], references=[label], rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=True)
      rouge_scores.append(score)

  # mean
  mean_rouge1 = sum([score['rouge1'] for score in rouge_scores]) / len(rouge_scores)
  mean_rouge2 = sum([score['rouge2'] for score in rouge_scores]) / len(rouge_scores)
  mean_rougeL = sum([score['rougeL'] for score in rouge_scores]) / len(rouge_scores)

  print(f'R-1: {mean_rouge1:.4f}', f',R-2: {mean_rouge2:.4f}', f',R-L: {mean_rougeL:.4f}')



# one good and one bad
test OpenI

In [ ]:
from rouge import Rouge

rouge = Rouge()

interactive = True
interactive_times = 20
rouge_thre = 0.85
near_k_samples = 18
rouge_type = "rouge-1"
test_fold = 0

df_test_csv = pd.read_csv('Path of test_data.csv'.format(test_fold, test_fold))
train_data = pd.read_csv("Path of train_data.csv".format(test_fold, test_fold))
test_label_space = pd.read_csv("Path of test_label.csv".format(test_fold, test_fold))
train_label_space = pd.read_csv("Path of train_label.csv".format(test_fold, test_fold))

test_len = len(df_test_csv)
test_label_study_ids = []
for index, row in test_label_space.iterrows():
  test_label_study_ids.append(row["ids"])

# use_near_sample = False
# use_near_sample = True

#chestGPT_summary_near_sample_with_interactive(text, near_samples=None, interactive=False, former_good_response=None, former_bad_response=None):


def process_row_with_random_choice_samples(args):
    row_index, row = args

    test_id = row['ids']
    test_findings, test_impression = row["finding"], row["impression\n"]

    test_sample_label = np.array(test_label_space.iloc[test_label_study_ids.index(test_id)][1:])

    near_study_ids = random_sample_selection_v2_for_openI(test_sample_label, train_label_space, m=test_len, k=near_k_samples)

    near_samples = []
    for index, train_row in train_data.iterrows():
      if len(near_study_ids) == 0:
        break
      if int(train_row["ids"]) in near_study_ids:
        near_samples.append({"finding": train_data.iloc[index]["finding"], "impression": train_data.iloc[index]["impression\n"]})
        near_study_ids.remove(train_row["ids"])

    good_reponse = []
    bad_reponse = []
    former_socre = 0
    former_bad_socre = 0
    all_response_score, all_response = [], []
    try_count = 0
    while True:
      if len(good_reponse) == 0 and len(bad_reponse) == 0:
        reponse = chestGPT_summary_near_sample_with_interactive_v4(test_findings, near_samples=near_samples)
      else:
        reponse = chestGPT_summary_near_sample_with_interactive_v4(test_findings, near_samples=near_samples, interactive=interactive, former_good_response=good_reponse, former_bad_response=bad_reponse)
     
      fotmatted_response = reponse.replace("IMPRESSION:", "")
      
      compare_scores = []
      for near_sa in near_samples:
        scores = rouge.get_scores([fotmatted_response], [near_sa["impression"]])
        single_score = scores[0][rouge_type]["f"]
        compare_scores.append(single_score)
      # print(compare_scores)
      score = np.mean(np.array(compare_scores))

      all_response_score.append(score)
      all_response.append(fotmatted_response)
      try_count += 1
      if score >= rouge_thre and score > former_socre:
        former_socre = score
        good_reponse.clear()
        good_reponse.append({"summarize": fotmatted_response, "score": score})
      if score < rouge_thre - 0.1 and score < former_socre:
        former_bad_socre = score
        bad_reponse.clear()
        bad_reponse.append({"summarize": fotmatted_response, "score": score})
      if try_count > interactive_times:
        break

    max_score_index = all_response_score.index(max(all_response_score))
    max_score = all_response_score[max_score_index]
    final_best_response = all_response[max_score_index]
    # print("max score=", max_score, "\n", final_best_response)
    # print("max score=", max_score, "\n")
    return row_index, final_best_response

# parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
  results = dict(tqdm(executor.map(process_row_with_random_choice_samples, df_test_csv.iterrows()), total=test_len))

# same order as the input
df_test_csv['summary'] = df_test_csv.index.map(results.get)
# df_test_csv['summary'] = df_test_csv.index.map(dict(results))

out_file_name = "fold{}_IntactV4_near{}_Tt_1Gd1Bd_i{}r{}_{}.csv".format(test_fold, near_k_samples, interactive_times, rouge_thre, rouge_type)
output_file = "Root Path/{}".format(out_file_name)
df_test_csv.to_csv(output_file, index=False)


rouge_e = evaluate.load('rouge')

rouge_scores = []
for index, row in df_test_csv.iterrows():
    label = row['impression\n']
    prediction = row['summary']
    prediction = prediction.replace("IMPRESSION:", "")
    score = rouge_e.compute(predictions=[prediction], references=[label], rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores.append(score)


# mean
mean_rouge1 = sum([score['rouge1'] for score in rouge_scores]) / len(rouge_scores)
mean_rouge2 = sum([score['rouge2'] for score in rouge_scores]) / len(rouge_scores)
mean_rougeL = sum([score['rougeL'] for score in rouge_scores]) / len(rouge_scores)

print(out_file_name)
print(f'R-1: {mean_rouge1:.4f}', f'R-2: {mean_rouge2:.4f}', f'R-L: {mean_rougeL:.4f}')



# **One Good And N Bad**

# Mimic data 

In [ ]:
import os.path as osp
import pandas as pd

root = "Root path of saving CSV"

# MIMIC-CXR offical training split.
train_data = pd.read_csv("Path of mimic_report_sections_train_v4.csv") 
# CheXpert is used to obtain the train data labels.
train_label_space = pd.read_csv('Path of train_v4_labels.csv')

# MIMIC-CXR offical testing split. CheXpert is used to obtain the label.
test_label_space = pd.read_csv('Path of test_v6_labels.csv')


test_label_study_ids = []
for index, row in test_label_space.iterrows():
  test_label_study_ids.append(row["study_id"])


# Pre-defined parameters
interactive = True
interactive_times = 17
rouge_thre = 0.7
one_near_k_samples = 200
two_near_k_samples = 14
rouge_type = "rouge-1"


print("Start at", time.strftime("%Y-%m-%d %H:%M:%S"))

def process_row_with_random_choice_samples(args):
    row_index, row = args

    test_subject_id, test_study_id = row['subject_id'], row['study_id']
    test_findings, test_impression = row["findings"], row["impression"]

    # Test label vector
    test_sample_label = np.array(test_label_space.iloc[test_label_study_ids.index(test_study_id)][2:])

    # Similar Search. Stage-1
    # Return Top-<one_near_k_samples> samples' id based on label vector.  m is the size of our corpus. randomly sample from all training data.
    near_study_ids = random_sample_selection_v2(test_sample_label, train_label_space, m=10000, k=one_near_k_samples)

    # Get the similar report Findings and Impression based on 'near_study_ids'
    near_samples = []
    for index, train_row in train_data.iterrows():
      if len(near_study_ids) == 0:
        break
      if int(train_row["study_id"]) in near_study_ids:
        near_samples.append({"finding": train_data.iloc[index]["finding"], "impression": train_data.iloc[index]["impression"]})
        near_study_ids.remove(train_row["study_id"])

    # Similar Search. Stage-2
    # Choose Top-<two_near_k_samples> samples from [near_samples] based on Rouge-1 score compared with test findings.
    similar_samples = []
    similar_samples_score = []
    similar_score = []
    for sample in near_samples:
      s_scores = rouge.get_scores([test_findings], [sample["finding"]])
      similar_score.append(s_scores[0][rouge_type]["f"])

    np_similar_score = np.array(similar_score)
    similar_scores_index = np.argsort(-np_similar_score)
    choose_index = similar_scores_index[:two_near_k_samples]

    # Now we have some [similar_samples] filterd by two-stage Similar Search.
    for c_index in choose_index:
      similar_samples.append(near_samples[c_index])

    good_reponse = []
    bad_reponse = []
    former_socre = 0
    former_bad_socre = 0
    all_response_score, all_response = [], []
    try_count = 0
    while True:
      if len(good_reponse) == 0 and len(bad_reponse) == 0:
        reponse = chestGPT_summary_near_sample_with_interactive_v6(test_findings, near_samples=similar_samples)
      else:
        reponse = chestGPT_summary_near_sample_with_interactive_v6(test_findings, near_samples=similar_samples, interactive=interactive, former_good_response=good_reponse, former_bad_response=bad_reponse)

      if reponse == -2:
        print("exceed length, pop 2 similar examples")
        # list_len = len(similar_samples)
        for i in range(2):
          similar_samples.pop()
        continue

      fotmatted_response = reponse.replace("IMPRESSION:", "")
      
      compare_scores = []
      for near_sa in similar_samples:
        scores = rouge.get_scores([fotmatted_response], [near_sa["impression"]])
        single_score = scores[0][rouge_type]["f"]
        compare_scores.append(single_score)

      # print(compare_scores)
      score = np.mean(np.array(compare_scores))
      
      all_response_score.append(score)
      all_response.append(fotmatted_response)
      try_count += 1
      # if it is better than former
      if score >= rouge_thre and score > former_socre:
        former_socre = score
        good_reponse.clear()  # Only keep one good response
        good_reponse.append({"summarize": fotmatted_response, "score": score})
      # if it is worse than former
      if score < rouge_thre and score < former_socre:
        former_bad_socre = score
        # Keep more than one bad response. Length limit is 8 here.
        if len(bad_reponse) > 8: 
          bad_reponse = bad_reponse[-8:]    
        # bad_reponse.clear()  
        bad_reponse.append({"summarize": fotmatted_response, "score": score})
      if try_count > interactive_times:
        break

    max_score_index = all_response_score.index(max(all_response_score))
    max_score = all_response_score[max_score_index]
    final_best_response = all_response[max_score_index]
    # print("max score=", max_score, "\n", final_best_response)
    # print("max score=", max_score, "\n")
    return row_index, final_best_response


# 'mid_path' is the experiment name. InteractV6: Prompt version. 2Snear: Two-stage similar selection.  
mid_path = "InteractV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}".format(one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)
save_root = osp.join(root, mid_path)
if not osp.exists(save_root):
  os.makedirs(save_root) 
print(save_root)

# To avoid accidental interruption of the program and loss of results, I split it into multiple patches.
for loop_index in range(7):
  print(time.strftime("%Y-%m-%d %H:%M:%S"))
  Test_start = loop_index * 250
  Test_end = (loop_index + 1) * 250

  if Test_end > len(test_label_space):
    Test_end = len(test_label_space)
  print(Test_start, Test_end)

  # load test data
  df_test_csv = pd.read_csv('baseline_sections_test_V2_Clean.csv')
  df_test_csv = df_test_csv[Test_start:Test_end]

  # parallel
  with concurrent.futures.ThreadPoolExecutor() as executor:
    # time.sleep(20)
    results = dict(tqdm(executor.map(process_row_with_random_choice_samples, df_test_csv.iterrows()), total=len(df_test_csv)))

  # same order as the input
  df_test_csv['summary'] = df_test_csv.index.map(results.get)

  # out_file_name is name of split csv
  out_file_name = "test{}_{}_IntactV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}.csv".format(Test_start, Test_end, one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)
  output_file = osp.join(save_root, "{}".format(out_file_name))
  df_test_csv.to_csv(output_file, index=False)

  rouge_e = evaluate.load('rouge')

  rouge_scores = []
  for index, row in df_test_csv.iterrows():
      label = row['impression']
      prediction = row['summary']
      prediction = prediction.replace("IMPRESSION:", "")
      score = rouge_e.compute(predictions=[prediction], references=[label], rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=True)
      rouge_scores.append(score)

  # mean
  mean_rouge1 = sum([score['rouge1'] for score in rouge_scores]) / len(rouge_scores)
  mean_rouge2 = sum([score['rouge2'] for score in rouge_scores]) / len(rouge_scores)
  mean_rougeL = sum([score['rougeL'] for score in rouge_scores]) / len(rouge_scores)

  print(f'R-1: {mean_rouge1:.4f}', f',R-2: {mean_rouge2:.4f}', f',R-L: {mean_rougeL:.4f}')


# Final test for all csv data

print("### Final Test for All CSVs ###")

rouge_e = evaluate.load('rouge')

contact_list = []
range_list = [0, 250, 500, 750, 1000, 1250, 1500, 1603]
for loop_index in range(len(range_list)-1):
  Test_start = range_list[loop_index]
  Test_end = range_list[loop_index + 1]
  out_file_name = "test{}_{}_IntactV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}.csv".format(Test_start, Test_end, one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)
  test_pd = pd.read_csv(osp.join(save_root, out_file_name))
  contact_list.append(test_pd)

df_save_csv = pd.concat(contact_list)
df_save_csv.to_csv(osp.join(save_root, "testAll_IntactV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}.csv".format(one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)), index=False)


rouge_scores = []
for index, row in df_save_csv.iterrows():
    label = row['impression']
    prediction = row['summary']
    prediction = prediction.replace("IMPRESSION:", "")
    score = rouge_e.compute(predictions=[prediction], references=[label], rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores.append(score)


# mean
mean_rouge1 = sum([score['rouge1'] for score in rouge_scores]) / len(rouge_scores)
mean_rouge2 = sum([score['rouge2'] for score in rouge_scores]) / len(rouge_scores)
mean_rougeL = sum([score['rougeL'] for score in rouge_scores]) / len(rouge_scores)

print(f'R-1: {mean_rouge1:.4f}', f'R-2: {mean_rouge2:.4f}', f'R-L: {mean_rougeL:.4f}')



# **One Good And N Bad**

# OpenI data 


In [ ]:
from rouge import Rouge

rouge = Rouge()

root = ""

train_data = pd.read_csv("train576_data.csv")
test_label_space = pd.read_csv("test576_label.csv")
train_label_space = pd.read_csv("train576_label.csv")

train_len = len(train_label_space)
test_label_study_ids = []
for index, row in test_label_space.iterrows():
  test_label_study_ids.append(row["ids"])

# use_near_sample = False
# use_near_sample = True
interactive = True
interactive_times = 20
rouge_thre = 0.85
one_near_k_samples = 200
two_near_k_samples = 16
rouge_type = "rouge-1"


#chestGPT_summary_near_sample_with_interactive(text, near_samples=None, interactive=False, former_good_response=None, former_bad_response=None):
print("Start at", time.strftime("%Y-%m-%d %H:%M:%S"))
def process_row_with_random_choice_samples(args):
    row_index, row = args

    test_id = row['ids']
    test_findings, test_impression = row["finding"], row["impression\n"]

    test_sample_label = np.array(test_label_space.iloc[test_label_study_ids.index(test_id)][1:])

    near_study_ids = random_sample_selection_v2_for_openI(test_sample_label, train_label_space, m=train_len, k=one_near_k_samples)

    near_samples = []
    for index, train_row in train_data.iterrows():
      if len(near_study_ids) == 0:
        break
      if int(train_row["ids"]) in near_study_ids:
        near_samples.append({"finding": train_data.iloc[index]["finding"], "impression": train_data.iloc[index]["impression\n"]})
        near_study_ids.remove(train_row["ids"])


    similar_samples = []
    similar_samples_score = []
    similar_score = []
    for sample in near_samples:
      # s_scores = rouge_e.compute(predictions=[test_findings], references=[sample["finding"]], rouge_types=[rouge_type], use_stemmer=True)
      # similar_score.append(s_scores[rouge_type])
      
      s_scores = rouge.get_scores([test_findings], [sample["finding"]])
      similar_score.append(s_scores[0][rouge_type]["f"])

    np_similar_score = np.array(similar_score)
    similar_scores_index = np.argsort(-np_similar_score)
    choose_index = similar_scores_index[:two_near_k_samples]

    for c_index in choose_index:
      similar_samples.append(near_samples[c_index])


    good_reponse = []
    bad_reponse = []
    former_socre = 0
    former_bad_socre = Math.inf
    all_response_score, all_response = [], []
    try_count = 0
    while True:
      if len(good_reponse) == 0 and len(bad_reponse) == 0:
        reponse = chestGPT_summary_near_sample_with_interactive_v6(test_findings, near_samples=similar_samples)
      else:
        reponse = chestGPT_summary_near_sample_with_interactive_v6(test_findings, near_samples=similar_samples, interactive=interactive, former_good_response=good_reponse, former_bad_response=bad_reponse)

      if reponse == -2:
        print("exceed length, pop the similar examples")
        # list_len = len(similar_samples)
        for i in range(2):
          similar_samples.pop()
        continue
      fotmatted_response = reponse.replace("IMPRESSION:", "")
      
      compare_scores = []
      for near_sa in similar_samples:
        scores = rouge.get_scores([fotmatted_response], [near_sa["impression"]])
        single_score = scores[0][rouge_type]["f"]
        compare_scores.append(single_score)

      # print(compare_scores)
      score = np.mean(np.array(compare_scores))
      
      all_response_score.append(score)
      all_response.append(fotmatted_response)
      try_count += 1
      if score >= rouge_thre and score > former_socre:
        former_socre = score
        good_reponse.clear()
        good_reponse.append({"summarize": fotmatted_response, "score": score})
      # if score < rouge_thre - 0.1 and score < former_socre:
      if score < rouge_thre and score < former_bad_socre:
        former_bad_socre = score
        if len(bad_reponse) > 8:
          bad_reponse = bad_reponse[-8:]    
        # bad_reponse.clear()
        bad_reponse.append({"summarize": fotmatted_response, "score": score})
      if try_count > interactive_times:
        break

    max_score_index = all_response_score.index(max(all_response_score))
    max_score = all_response_score[max_score_index]
    final_best_response = all_response[max_score_index]
    # print("max score=", max_score, "\n", final_best_response)
    # print("max score=", max_score, "\n")
    return row_index, final_best_response


mid_path = "InteractV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}".format(one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)
save_root = osp.join(root, mid_path)
if not osp.exists(save_root):
  os.makedirs(save_root) 
print(save_root)


for loop_index in range(3):
  print(time.strftime("%Y-%m-%d %H:%M:%S"))
  Test_start = loop_index * 200
  Test_end = (loop_index + 1) * 200

  df_test_csv = pd.read_csv("test576_data.csv")
  if Test_end > len(df_test_csv):
    Test_end = len(df_test_csv)

  print(Test_start, Test_end)
  df_test_csv = df_test_csv[Test_start:Test_end]

  # parallel
  with concurrent.futures.ThreadPoolExecutor() as executor:
    # time.sleep(20)
    results = dict(tqdm(executor.map(process_row_with_random_choice_samples, df_test_csv.iterrows()), total=len(df_test_csv)))

  # same order as the input
  df_test_csv['summary'] = df_test_csv.index.map(results.get)

  out_file_name = "test{}_{}_IntactV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}.csv".format(Test_start, Test_end, one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)
  output_file = osp.join(save_root, "{}".format(out_file_name))
  df_test_csv.to_csv(output_file, index=False)

  rouge_e = evaluate.load('rouge')

  rouge_scores = []
  for index, row in df_test_csv.iterrows():
      label = row['impression\n']
      prediction = row['summary']
      prediction = prediction.replace("IMPRESSION:", "")
      score = rouge_e.compute(predictions=[prediction], references=[label], rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=True)
      rouge_scores.append(score)

  # mean
  mean_rouge1 = sum([score['rouge1'] for score in rouge_scores]) / len(rouge_scores)
  mean_rouge2 = sum([score['rouge2'] for score in rouge_scores]) / len(rouge_scores)
  mean_rougeL = sum([score['rougeL'] for score in rouge_scores]) / len(rouge_scores)

  print(f'R-1: {mean_rouge1:.4f}', f',R-2: {mean_rouge2:.4f}', f',R-L: {mean_rougeL:.4f}')



# Final test for all csv data

print("### Final Test for All CSVs ###")

rouge_e = evaluate.load('rouge')

contact_list = []
range_list = [0, 200, 400, 576]
for loop_index in range(len(range_list)-1):
  Test_start = range_list[loop_index]
  Test_end = range_list[loop_index + 1]
  out_file_name = "test{}_{}_IntactV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}.csv".format(Test_start, Test_end, one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)
  test_pd = pd.read_csv(osp.join(save_root, out_file_name))
  contact_list.append(test_pd)

df_save_csv = pd.concat(contact_list)
df_save_csv.to_csv(osp.join(save_root, "testAll_IntactV6_2Snear{}-{}_1Gd8Bds_i{}r{}_{}.csv".format(one_near_k_samples, two_near_k_samples, interactive_times, rouge_thre, rouge_type)), index=False)


rouge_scores = []
for index, row in df_save_csv.iterrows():
    label = row['impression']
    prediction = row['summary']
    prediction = prediction.replace("IMPRESSION:", "")
    score = rouge_e.compute(predictions=[prediction], references=[label], rouge_types=["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores.append(score)


# mean
mean_rouge1 = sum([score['rouge1'] for score in rouge_scores]) / len(rouge_scores)
mean_rouge2 = sum([score['rouge2'] for score in rouge_scores]) / len(rouge_scores)
mean_rougeL = sum([score['rougeL'] for score in rouge_scores]) / len(rouge_scores)

print(f'R-1: {mean_rouge1:.4f}', f'R-2: {mean_rouge2:.4f}', f'R-L: {mean_rougeL:.4f}')

